# Geração de Perfis Cognitivos para Simulação SINKT

Este notebook implementa a **Etapa 1** do pipeline SINKT: criação de perfis cognitivos fundamentados em modelos clássicos de Knowledge Tracing.

## Objetivo
Criar 6 perfis cognitivos distintos que representam diferentes tipos de estudantes, cada um com características únicas de aprendizagem.

## Saída
- `data/output/notebooks/geracao_perfis/profiles.json`: Arquivo JSON contendo todos os perfis cognitivos
- `data/output/notebooks/geracao_perfis/profiles.md`: Documentação em Markdown dos perfis gerados

## Importação de Bibliotecas

Carregamento das bibliotecas necessárias para manipulação de dados e validação.

In [1]:
import json
import os
from datetime import datetime
from typing import Dict, List, Any
import numpy as np

## Definição dos Perfis Cognitivos

Cada perfil é caracterizado por 9 parâmetros fundamentais:
- **mastery_init_level**: Nível inicial de domínio (0-1)
- **learn_rate**: Taxa de aprendizagem
- **slip**: Probabilidade de erro quando sabe
- **guess**: Probabilidade de acerto quando não sabe
- **logic_skill**: Habilidade lógica
- **reading_skill**: Habilidade de leitura
- **tech_familiarity**: Familiaridade com tecnologia
- **memory_capacity**: Capacidade de retenção de memória
- **learning_consistency**: Consistência e regularidade nos estudos

In [2]:
profiles_data = [
    {
        "id": "balanced",
        "nome": "Estudante Equilibrado",
        "descricao": "Perfil médio com habilidades balanceadas em todas as dimensões. Representa o estudante típico com desenvolvimento uniforme.",
        "mastery_init_level": 0.55,
        "learn_rate": 0.035,
        "slip": 0.15,
        "guess": 0.15,
        "logic_skill": 0.55,
        "reading_skill": 0.55,
        "tech_familiarity": 0.55,
        "memory_capacity": 0.55,
        "learning_consistency": 0.60
    },
    {
        "id": "quick_learner",
        "nome": "Aprendiz Rápido",
        "descricao": "Alta capacidade de aprendizagem e domínio inicial. Adquire conceitos rapidamente mas pode cometer erros por pressa.",
        "mastery_init_level": 0.50,
        "learn_rate": 0.08,
        "slip": 0.18,
        "guess": 0.12,
        "logic_skill": 0.7,
        "reading_skill": 0.65,
        "tech_familiarity": 0.8,
        "memory_capacity": 0.80,
        "learning_consistency": 0.85
    },
    {
        "id": "careful",
        "nome": "Estudante Cuidadoso",
        "descricao": "Baixa taxa de erros quando sabe o conteúdo, mas aprende de forma mais gradual. Meticuloso e detalhista.",
        "mastery_init_level": 0.45,
        "learn_rate": 0.025,
        "slip": 0.08,
        "guess": 0.1,
        "logic_skill": 0.6,
        "reading_skill": 0.7,
        "tech_familiarity": 0.5,
        "memory_capacity": 0.70,
        "learning_consistency": 0.90
    },
    {
        "id": "struggling",
        "nome": "Estudante com Dificuldades",
        "descricao": "Baixo domínio inicial e lenta aprendizagem. Pode precisar de mais apoio e tempo para absorver conceitos.",
        "mastery_init_level": 0.25,
        "learn_rate": 0.015,
        "slip": 0.25,
        "guess": 0.2,
        "logic_skill": 0.3,
        "reading_skill": 0.35,
        "tech_familiarity": 0.2,
        "memory_capacity": 0.40,
        "learning_consistency": 0.40
    },
    {
        "id": "logical",
        "nome": "Pensador Lógico",
        "descricao": "Excelente habilidade lógica mas com dificuldades em leitura. Excelente em problemas algorítmicos e matemáticos.",
        "mastery_init_level": 0.5,
        "learn_rate": 0.04,
        "slip": 0.12,
        "guess": 0.08,
        "logic_skill": 0.85,
        "reading_skill": 0.3,
        "tech_familiarity": 0.6,
        "memory_capacity": 0.60,
        "learning_consistency": 0.75
    },
    {
        "id": "intuitive",
        "nome": "Estudante Intuitivo",
        "descricao": "Boa intuição e habilidade de leitura, mas dificuldade com formalismo lógico. Aprende por exemplo e prática.",
        "mastery_init_level": 0.4,
        "learn_rate": 0.045,
        "slip": 0.16,
        "guess": 0.18,
        "logic_skill": 0.35,
        "reading_skill": 0.8,
        "tech_familiarity": 0.45,
        "memory_capacity": 0.65,
        "learning_consistency": 0.65
    }
]

## Validação dos Perfis

Verifica se todos os parâmetros estão dentro dos limites válidos (0-1) e se há diversidade suficiente entre os perfis.

In [3]:
def validate_profiles(profiles: List[Dict[str, Any]]) -> List[str]:
    """Valida os perfis cognitivos."""
    errors = []
    
    required_fields = ['id', 'nome', 'descricao', 'mastery_init_level', 'learn_rate', 
                       'slip', 'guess', 'logic_skill', 'reading_skill', 'tech_familiarity',
                       'memory_capacity', 'learning_consistency']
    
    numeric_fields = ['mastery_init_level', 'learn_rate', 'slip', 'guess', 
                      'logic_skill', 'reading_skill', 'tech_familiarity',
                      'memory_capacity', 'learning_consistency']
    
    profile_ids = set()
    
    for i, profile in enumerate(profiles):
        # Verifica campos obrigatórios
        for field in required_fields:
            if field not in profile:
                errors.append(f"Perfil {i}: Campo '{field}' ausente")
        
        # Verifica ID único
        if 'id' in profile:
            if profile['id'] in profile_ids:
                errors.append(f"Perfil {i}: ID '{profile['id']}' duplicado")
            profile_ids.add(profile['id'])
        
        # Verifica ranges dos parâmetros numéricos
        for field in numeric_fields:
            if field in profile:
                value = profile[field]
                if not isinstance(value, (int, float)):
                    errors.append(f"Perfil {profile.get('id', i)}: '{field}' deve ser numérico")
                elif value < 0 or value > 1:
                    errors.append(f"Perfil {profile.get('id', i)}: '{field}' = {value} fora do range [0, 1]")
    
    return errors

# Executar validação
validation_errors = validate_profiles(profiles_data)

if validation_errors:
    print("❌ Erros de validação encontrados:")
    for error in validation_errors:
        print(f"  - {error}")
else:
    print("✅ Todos os perfis são válidos!")

✅ Todos os perfis são válidos!


## Cálculo de Estatísticas dos Perfis

Calcula estatísticas descritivas para cada parâmetro dos perfis, permitindo visualizar a diversidade do conjunto.

In [4]:
def calculate_profile_statistics(profiles: List[Dict[str, Any]]) -> Dict[str, Dict[str, float]]:
    """Calcula estatísticas dos parâmetros dos perfis."""
    numeric_fields = ['mastery_init_level', 'learn_rate', 'slip', 'guess', 
                      'logic_skill', 'reading_skill', 'tech_familiarity',
                      'memory_capacity', 'learning_consistency']
    
    stats = {}
    
    for field in numeric_fields:
        values = [p[field] for p in profiles if field in p]
        if values:
            stats[field] = {
                'mean': float(np.mean(values)),
                'std': float(np.std(values)),
                'min': float(np.min(values)),
                'max': float(np.max(values))
            }
    
    return stats

# Calcular e exibir estatísticas
stats = calculate_profile_statistics(profiles_data)

print("\n📊 Estatísticas dos Parâmetros:\n")
for param, values in stats.items():
    print(f"  {param}:")
    print(f"    Média: {values['mean']:.3f}")
    print(f"    Desvio: {values['std']:.3f}")
    print(f"    Range: [{values['min']:.3f}, {values['max']:.3f}]")
    print()


📊 Estatísticas dos Parâmetros:

  mastery_init_level:
    Média: 0.442
    Desvio: 0.098
    Range: [0.250, 0.550]

  learn_rate:
    Média: 0.040
    Desvio: 0.020
    Range: [0.015, 0.080]

  slip:
    Média: 0.157
    Desvio: 0.052
    Range: [0.080, 0.250]

  guess:
    Média: 0.138
    Desvio: 0.043
    Range: [0.080, 0.200]

  logic_skill:
    Média: 0.558
    Desvio: 0.190
    Range: [0.300, 0.850]

  reading_skill:
    Média: 0.558
    Desvio: 0.181
    Range: [0.300, 0.800]

  tech_familiarity:
    Média: 0.517
    Desvio: 0.180
    Range: [0.200, 0.800]

  memory_capacity:
    Média: 0.617
    Desvio: 0.125
    Range: [0.400, 0.800]

  learning_consistency:
    Média: 0.692
    Desvio: 0.167
    Range: [0.400, 0.900]



## Criação do Arquivo JSON de Saída

Estrutura e salva os perfis em formato JSON com metadados completos.

In [5]:
# Criar estrutura completa com metadados
output_data = {
    "metadata": {
        "description": "Conjunto de perfis cognitivos para simulação de estudantes SINKT",
        "version": "1.0.0",
        "created_at": datetime.now().isoformat(),
        "num_profiles": len(profiles_data)
    },
    "profiles": {profile['id']: profile for profile in profiles_data}
}

# Criar diretório de saída se não existir
os.makedirs('data/output/notebooks/geracao_perfis', exist_ok=True)

# Salvar arquivo JSON
output_file = 'data/output/notebooks/geracao_perfis/profiles.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(output_data, f, indent=2, ensure_ascii=False)

print(f"✅ Perfis salvos em: {output_file}")
print(f"📦 Total de perfis: {len(profiles_data)}")

✅ Perfis salvos em: data/output/notebooks/geracao_perfis/profiles.json
📦 Total de perfis: 6


## Resumo da Execução

Exibe um resumo final com informações sobre os perfis gerados.

In [6]:
print("\n" + "="*60)
print("🎉 GERAÇÃO DE PERFIS CONCLUÍDA COM SUCESSO!")
print("="*60)
print(f"\n📁 Arquivos gerados:")
print(f"  - {output_file}")
print(f"\n📊 Perfis criados:")
for profile in profiles_data:
    print(f"  - {profile['nome']} ({profile['id']})")
print("\n✅ Próximo passo: Execute o notebook '02_geracao_estudantes.ipynb'")
print("="*60)


🎉 GERAÇÃO DE PERFIS CONCLUÍDA COM SUCESSO!

📁 Arquivos gerados:
  - data/output/notebooks/geracao_perfis/profiles.json

📊 Perfis criados:
  - Estudante Equilibrado (balanced)
  - Aprendiz Rápido (quick_learner)
  - Estudante Cuidadoso (careful)
  - Estudante com Dificuldades (struggling)
  - Pensador Lógico (logical)
  - Estudante Intuitivo (intuitive)

✅ Próximo passo: Execute o notebook '02_geracao_estudantes.ipynb'
